# 完全なビジネスソリューション

## 今、私たちは1日目から次のレベルにプロジェクトを取ります

### ビジネスチャレンジ：

将来のクライアント、投資家、潜在的な新兵に使用される企業のパンフレットを構築する製品を作成します。

会社名とその主要なウェブサイトが提供されます。

実際のビジネスアプリケーションの例については、このノートブックの終わりを参照してください。

そして覚えておいてください：あなたが問題やアイデアがあるならば、私はいつでも利用可能です！手を差し伸べてください。

In [ ]:
# 輸入
# これらが失敗した場合は、コマンドプロンプトに（LLMS）がある「アクティブ化された」環境から実行していることを確認してください

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
# 初期化と定数

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("APIキーはこれまでのところ良く見えます")
else:
    print("あなたのAPIキーに問題があるかもしれませんか？トラブルシューティングノートブックにアクセスしてください！")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [ ]:
# ウェブページを表すクラス

# 一部のWebサイトでは、適切なヘッダーを使用するときに使用する必要があります。
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [ ]:
ed = Website("https://edwarddonner.com")
ed.links

## 最初のステップ：GPT-4O-MINIにどのリンクが関連しているかを把握してもらう

### GPT-4O-MINIへの電話を使用して、Webページのリンクを読み取り、構造化されたJSONで応答します。  
どのリンクが関連するかを決定し、「/about」などの相対的なリンクを「https://company.com/about」に置き換える必要があります。  
プロンプトでどのように応答するかの例を提供する「ワンショットプロンプト」を使用します。

これは、微妙な理解を必要とするため、LLMにとって優れたユースケースです。 Webページを解析して分析して、LLMSなしでこれをコーディングしようとすることを想像してください - それは非常に難しいでしょう！

サイドノート：「構造化された出力」と呼ばれるより高度な手法があり、スペックに従って応答するためにモデルが必要です。私たちは、自律剤AIプロジェクト中に8週目にこの手法をカバーしています。

In [ ]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [ ]:
print(link_system_prompt)

In [ ]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [ ]:
print(get_links_user_prompt(ed))

In [ ]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [ ]:
# 人類は彼らのサイトを削るのを難しくしているので、私はハガーフェイスを使用しています。

huggingface = Website("https://huggingface.co")
huggingface.links

In [ ]:
get_links("https://huggingface.co")

## 2番目のステップ：パンフレットを作りましょう！

すべての詳細をGPT4-Oへの別のプロンプトに組み立てます

In [ ]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("発見されたリンク：", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details("https://huggingface.co"))

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# または、よりユーモラスなパンフレットのために以下の線を除外 - これは、「トーン」を組み込むことがどれほど簡単かを示しています。

# System_prompt = "あなたは、会社のWebサイトからいくつかの関連ページの内容を分析するアシスタントです\
# そして、見込み客、投資家、新兵のために、会社についてのユーモラスで面白い、ジョークのパンフレットを作成します。マークダウンで応答します。\
# 情報がある場合は、企業文化、顧客、キャリア/ジョブの詳細を含めてください。」


In [ ]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # 5,000文字以上の場合は切り捨てます
    return user_prompt

In [ ]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

In [ ]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("HuggingFace", "https://huggingface.co")

## 最後に - マイナーな改善

わずかな調整で、これを変更して、結果がOpenaiからストリーミングされるようにすることができます。
おなじみのタイプライターアニメーションを備えています

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

In [ ]:
# HuggingFaceのパンフレットを作成するときに、システムプロンプトをユーモラスなバージョンに変更してみてください。

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">ビジネスアプリケーション</h2>
            <span style="color:#181;">この演習では、1日目のコードを拡張して複数のLLM呼び出しを行い、ドキュメントを生成しました。

これはおそらく、LLMSへの複数の呼び出しを組み合わせたため、エージェントAI設計パターンの最初の例です。これにより、2週目にさらに登場し、完全に自律剤ソリューションを構築する際に、8週目にエージェントAIに大きな方法で戻ります。

この方法でコンテンツを生成することは、非常に一般的なユースケースの1つです。要約と同様に、これはあらゆるビジネス垂直に適用できます。マーケティングコンテンツを作成し、スペックから製品チュートリアルを生成し、パーソナライズされた電子メールコンテンツを作成します。コンテンツ生成をビジネスに適用する方法を調べて、自分自身を概念実証プロトタイプにしてみてください。他の学生がコミュニティコントリビューションズフォルダーで行ったことをご覧ください - 非常に多くの貴重なプロジェクト - それはワイルドです！</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">第2週(とても楽しいです)に進む前に</h2>
            <span style="color:#900;">第1週の終わりの課題については、第1週の演習ノートブックをご覧ください。Frontier APIの操作に関する基本的な練習ができ、第2週に向けてしっかりと準備できます。</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">2週目に移動する前に（これはたくさんの楽しみです）</h2>
            <span style="color:#f71;">1. コースのリソースは <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">こちら</a>から入手できます。<br/>
2. 私は LinkedIn に <a href="https://www.linkedin.com/in/eddonner/">こちら</a> から登録しています。コースを受講されている方と交流できるのを楽しみにしています。<br/>
3. Twitter も試していて、<a href="https://x.com/edwarddonner">@edwarddonner<a> でアカウントを作成しています。皆さんからやり方を教えていただければ幸いです。</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">最後に！特別なお願いがあります</h2>
            <span style="color:#090;">
                編集者から、Udemyでこのコースを受講生に評価してもらうと非常に大きな違いが出ると言われました。評価は、Udemyがこのコースを他の受講生に公開するかどうかを決める主な基準の一つだからです。もし評価していただけるなら、本当に嬉しいです！また、何かご不明な点がございましたら、いつでもed@edwarddonner.comまでご連絡ください。
            </span>
        </td>
    </tr>
</table>